In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("idowuadamo/students-performance-in-2024-jamb")

print("Path to dataset files:", path)

100%|██████████| 74.9k/74.9k [00:00<00:00, 39.8MB/s]

Extracting files...
Path to dataset files: /root/.cache/kagglehub/datasets/idowuadamo/students-performance-in-2024-jamb/versions/1


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error


df = pd.read_csv('/content/drive/MyDrive/jamb_exam_results.csv')

df.columns = df.columns.str.lower().str.replace(' ', '_')

df = df.drop(columns=['student_id'])

df = df.fillna(0)

train_data, temp_data = train_test_split(df, test_size=0.4, random_state=1)
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=1)


In [3]:
# Разделение признаков (X) и целевой переменной (y)
train_dicts = train_data.drop(columns=['jamb_score']).to_dict(orient='records')
val_dicts = val_data.drop(columns=['jamb_score']).to_dict(orient='records')
test_dicts = test_data.drop(columns=['jamb_score']).to_dict(orient='records')

# Использование DictVectorizer для преобразования данных
dv = DictVectorizer(sparse=True)

X_train = dv.fit_transform(train_dicts)
X_val = dv.transform(val_dicts)
X_test = dv.transform(test_dicts)

y_train = train_data['jamb_score'].values
y_val = val_data['jamb_score'].values
y_test = test_data['jamb_score'].values


In [4]:
# Обучение дерева решений
model = DecisionTreeRegressor(max_depth=1, random_state=1)
model.fit(X_train, y_train)

# Предсказания и оценка модели
y_pred = model.predict(X_val)
rmse = mean_squared_error(y_val, y_pred, squared=False)

print(f"RMSE на валидационной выборке: {rmse}")


RMSE на валидационной выборке: 44.85913145740971


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [5]:
from sklearn.ensemble import RandomForestRegressor

# Обучение случайного леса
rf_model = RandomForestRegressor(n_estimators=10, random_state=1, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Предсказания и оценка RMSE
y_pred_rf = rf_model.predict(X_val)
rmse_rf = mean_squared_error(y_val, y_pred_rf, squared=False)

print(f"RMSE случайного леса на валидационных данных: {rmse_rf:.2f}")


RMSE случайного леса на валидационных данных: 43.16


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [6]:
import numpy as np

rmse_values = []
n_estimators_range = range(10, 201, 10)

# Цикл по разным значениям n_estimators
for n in n_estimators_range:
    rf_model = RandomForestRegressor(n_estimators=n, random_state=1, n_jobs=-1)
    rf_model.fit(X_train, y_train)
    y_pred_rf = rf_model.predict(X_val)
    rmse = mean_squared_error(y_val, y_pred_rf, squared=False)
    rmse_values.append(rmse)

# Найдем значение n_estimators, после которого RMSE перестает улучшаться
rmse_values = np.array(rmse_values)
optimal_n_estimators = n_estimators_range[np.argmin(rmse_values)]

# Вывод результатов
for n, rmse in zip(n_estimators_range, rmse_values):
    print(f"n_estimators: {n}, RMSE: {rmse:.3f}")

print(f"Оптимальное значение n_estimators: {optimal_n_estimators}")


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'roo

n_estimators: 10, RMSE: 43.158
n_estimators: 20, RMSE: 41.790
n_estimators: 30, RMSE: 41.556
n_estimators: 40, RMSE: 41.076
n_estimators: 50, RMSE: 40.957
n_estimators: 60, RMSE: 40.774
n_estimators: 70, RMSE: 40.588
n_estimators: 80, RMSE: 40.503
n_estimators: 90, RMSE: 40.435
n_estimators: 100, RMSE: 40.365
n_estimators: 110, RMSE: 40.348
n_estimators: 120, RMSE: 40.302
n_estimators: 130, RMSE: 40.286
n_estimators: 140, RMSE: 40.263
n_estimators: 150, RMSE: 40.254
n_estimators: 160, RMSE: 40.200
n_estimators: 170, RMSE: 40.187
n_estimators: 180, RMSE: 40.136
n_estimators: 190, RMSE: 40.152
n_estimators: 200, RMSE: 40.138
Оптимальное значение n_estimators: 180


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [9]:
# Обучение случайного леса с указанными параметрами
rf_model = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1, n_jobs=-1)
rf_model.fit(X_train, y_train)

# Получение важности признаков
importances = rf_model.feature_importances_
feature_names = dv.get_feature_names_out()

# Находим самый важный признак
important_features = sorted(zip(importances, feature_names), reverse=True)
most_important_feature = important_features[0]

print(f"Самый важный признак: {most_important_feature[1]}, Важность: {most_important_feature[0]:.3f}")


Самый важный признак: study_hours_per_week, Важность: 0.254


In [8]:
import xgboost as xgb
from xgboost import DMatrix

# Создание DMatrix для train и validation
dtrain = xgb.DMatrix(X_train, label=y_train)
dval = xgb.DMatrix(X_val, label=y_val)

# Watchlist для валидации
watchlist = [(dtrain, 'train'), (dval, 'validation')]

# Определение параметров
def train_xgb(eta):
    xgb_params = {
        'eta': eta,
        'max_depth': 6,
        'min_child_weight': 1,
        'objective': 'reg:squarederror',
        'nthread': 8,
        'seed': 1,
        'verbosity': 1,
    }

    # Обучение модели
    model = xgb.train(params=xgb_params, dtrain=dtrain, num_boost_round=100, evals=watchlist, early_stopping_rounds=10, verbose_eval=False)
    return model

# Обучение моделей с разными eta
model_eta_03 = train_xgb(eta=0.3)
model_eta_01 = train_xgb(eta=0.1)

# Получение RMSE
rmse_eta_03 = model_eta_03.best_score
rmse_eta_01 = model_eta_01.best_score

print(f"RMSE при eta=0.3: {rmse_eta_03:.3f}")
print(f"RMSE при eta=0.1: {rmse_eta_01:.3f}")

# Определяем лучший eta
if rmse_eta_01 < rmse_eta_03:
    print("Лучшее значение eta: 0.1")
elif rmse_eta_01 > rmse_eta_03:
    print("Лучшее значение eta: 0.3")
else:
    print("Оба значения eta дают одинаковый результат.")


RMSE при eta=0.3: 40.688
RMSE при eta=0.1: 40.166
Лучшее значение eta: 0.1
